In [1]:
from queue import PriorityQueue

class Job:
    def __init__(self, job_id, deadline, penalty):
        self.job_id = job_id
        self.deadline = deadline
        self.penalty = penalty

    def __lt__(self, other):
        return self.penalty > other.penalty 

def job_sequencing_with_deadlines(jobs, max_deadline):
    job_queue = PriorityQueue()
    for job in jobs:
        job_queue.put(job)

    job_sequence = [-1] * max_deadline
    
    while not job_queue.empty():
        job = job_queue.get()
        for d in range(job.deadline - 1, -1, -1):
            if job_sequence[d] == -1:
                job_sequence[d] = job.job_id
                break

    return job_sequence

jobs = [Job("a", 2, 100), Job("b", 1, 19), Job("c", 2, 27), Job("d", 1, 25), Job("e", 3, 15)]
max_deadline = max(job.deadline for job in jobs)
scheduled_jobs = job_sequencing_with_deadlines(jobs, max_deadline)
print("Job sequence:", scheduled_jobs)


Job sequence: ['c', 'a', 'e']


In [2]:
from queue import PriorityQueue

class Job:
    def __init__(self, job_id , penalty, deadline,  processing_time):
        self.job_id = job_id
        self.deadline = deadline
        self.penalty = penalty
        self.processing_time = processing_time
        self.completion_time = -1  # Initialize completion time to -1

    def __lt__(self, other):
        return self.penalty > other.penalty 

def job_sequencing_with_deadlines(jobs, max_deadline):
    job_queue = PriorityQueue()
    for job in jobs:
        job_queue.put(job)

    job_sequence = [-1] * max_deadline
    total_penalty = 0
    current_time = 0
    
    while not job_queue.empty():
        job = job_queue.get()
        for d in range(job.deadline - 1, -1, -1):
            if job_sequence[d] == -1:
                job_sequence[d] = job.job_id
                job.completion_time = current_time + job.processing_time
                total_penalty += job.penalty
                current_time = job.completion_time
                break

    return job_sequence, total_penalty

# Example usage:
jobs = [
    Job("a", 5,1,1),
    Job("b", 10,3,2),
    Job("c", 6,2,1),
    Job("d", 3,1,1),

]

max_deadline = max(job.deadline for job in jobs)
scheduled_jobs, total_penalty = job_sequencing_with_deadlines(jobs, max_deadline)
print("Job sequence:", scheduled_jobs)
print("Total Penalty:", total_penalty)


Job sequence: ['a', 'c', 'b']
Total Penalty: 21
